#  Install the Required Libraries


In [1]:
!pip install transformers datasets accelerate evaluate torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Load the Dataset

In [2]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("SKNahin/bengali-transliteration-data")

# Split the dataset (80% train, 20% validation)
dataset = dataset["train"].train_test_split(test_size=0.2, seed=42)

# Separate train and validation sets
train_data = dataset["train"]
val_data = dataset["test"]

# Use only subsets of train and validation data
train_data = train_data.select(range(1500))  # Use first 2000 samples
val_data = val_data.select(range(300))      # Use first 400 samples

# Display a sample
print("Sample data:")
print(train_data[0])
print(f"Training data size: {len(train_data)}")
print(f"Validation data size: {len(val_data)}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/300 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/333k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5006 [00:00<?, ? examples/s]

Sample data:
{'bn': 'এটা কোনো পোস্ট হলো মিয়া আবাল', 'rm': 'eta kono post holo mia abal'}
Training data size: 1500
Validation data size: 300


In [3]:
train_data.shape

(1500, 2)

In [4]:
val_data.shape

(300, 2)

# Preprocessing with Tokenization

In [5]:
from transformers import AutoTokenizer

# Select a pre-trained model (you can experiment with these)
model_checkpoints = {
    "mT5": "google/mt5-small",
}

# Initialize tokenizer for mT5
selected_model = "mT5"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoints[selected_model])

# Tokenization function
def preprocess_function(batch):
    inputs = tokenizer(batch["bn"], truncation=True, max_length=64, padding="max_length")
    targets = tokenizer(batch["rm"], truncation=True, max_length=64, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

# Preprocess datasets
train_data = train_data.map(preprocess_function, batched=True, num_proc=4)
val_data = val_data.map(preprocess_function, batched=True, num_proc=4)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Map (num_proc=4):   0%|          | 0/1500 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/300 [00:00<?, ? examples/s]

# Data Formatting for PyTorch

In [6]:
import torch
from datasets import DatasetDict

# Define data collator to dynamically pad inputs and labels during training
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model_checkpoints[selected_model])

# Convert datasets to PyTorch format
train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_data.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


# Load Pre-trained Model

In [7]:
from transformers import AutoModelForSeq2SeqLM

# Load the pre-trained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints[selected_model])


pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

#  Define Training Pipeline

In [8]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import torch

# Training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="/results",
    evaluation_strategy="steps",
    eval_steps=1000,
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,  # Reduce epochs
    save_steps=1000,
    save_total_limit=2,
    logging_steps=500,
    predict_with_generate=False,  # Disable beam search during training
    fp16=torch.cuda.is_available(),  # Mixed precision
    logging_dir='./logs',
    gradient_accumulation_steps=2  # Simulate larger batch size
)


# Define the Trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer
)



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-dfd487a9357a>:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


# Model training

In [9]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: ERROR API key must be 40 characters long, yours was 37


...

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss,Validation Loss


TrainOutput(global_step=188, training_loss=0.0, metrics={'train_runtime': 385.8579, 'train_samples_per_second': 7.775, 'train_steps_per_second': 0.487, 'total_flos': 198281134080000.0, 'train_loss': 0.0, 'epoch': 2.0})

# Save the Model

In [10]:
model.save_pretrained("/banglish-to-bangla")
tokenizer.save_pretrained("/banglish-to-bangla")


('/banglish-to-bangla/tokenizer_config.json',
 '/banglish-to-bangla/special_tokens_map.json',
 '/banglish-to-bangla/spiece.model',
 '/banglish-to-bangla/added_tokens.json',
 '/banglish-to-bangla/tokenizer.json')

#  Predict with Beam Search

In [11]:
def predict(text, num_beams=4):
    inputs = tokenizer(text, return_tensors="pt", max_length=64, truncation=True).to("cuda")
    outputs = model.generate(
        inputs.input_ids,
        max_length=64,
        num_beams=num_beams,
        early_stopping=True
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

text = "ami test korei code disi..."
translated = predict(text)
print(f"Input: {text}")
print(f"Output: {translated}")


Input: ami test korei code disi...
Output: <0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x02>


# Prediction 2

In [15]:
# Test a sample Banglish input
text = "ami test korei code disi..."
translated = predict(text)
print(f"Input (Banglish): {text}")
print(f"Output (Bangla): {translated}")


Input (Banglish): ami test korei code disi...
Output (Bangla): <0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x02>


In [16]:
print(f"Output (Bangla): {translated.encode('utf-8').decode('utf-8')}")


Output (Bangla): <0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x04><0x02>
